# ACID

_atomicity, consistency, isolation, and durability_

In [ ]:
import lamindb as ln
import lamindb.schema as lns
from sqlalchemy.exc import IntegrityError
from pydantic import ValidationError
import pytest

ln.nb.header()

In [ ]:
with pytest.raises(ValidationError):
    user = lns.User(id="123", name="test")

This `User` object lacks two required fields, email and handle.

Hence, a `ValidationError` is raised.

Let us now try to fix this situation.

In [ ]:
user = lns.User(id="123", name="test", email="abc@abc.com", handle="xyz")

In [ ]:
ln.add(user)

In [ ]:
ln.delete(user)